Install required libraries

In [1]:
#After executing the cell,please RESTART the kernel and run all the cells.
!pip install --user"ibm-watsonx-ai==1.0.4"
!pip install  --user "langchain==0.2.1" 
!pip install  --user "langchain-ibm==0.1.7"
!pip install  --user "langchain-community==0.2.1"
#!pip install --user "chromadb==0.4.24"
!pip install --user "chromadb==0.3.29"
#!pip install --upgrade chromadb
!pip install  --user "faiss-cpu==1.8.0"


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --useribm-watsonx-ai

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


Loading file

In [1]:
#!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BYlUHaillwM8EUItaIytHQ/companypolicies.txt"
from langchain_community.document_loaders import TextLoader
loader = TextLoader("companypolicies.txt")
data = loader.load()
#data[0]

Splitting data

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len,
)

chunks = text_splitter.split_documents(data)
len(chunks)

215

Embedding model

In [3]:
import os

os.environ['WATSONX_APIKEY'] = 'anJE64S_680sJPkVOd64EVj6SDKpMYHFZ5_IIe3u3sQb'


In [4]:
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings

embed_params = {
    EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3,
    EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True},
}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/slate-125m-english-rtrvr",
     url="https://eu-de.ml.cloud.ibm.com",
    project_id="7ce63394-6c4b-4e32-8909-456031209bab",
    params=embed_params,
)


Vector store Chroma

In [5]:
chunks[0:2]

[Document(metadata={'source': 'companypolicies.txt'}, page_content='1.\tCode of Conduct'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Our Code of Conduct outlines the fundamental principles and ethical standards that guide every')]

In [6]:
from langchain.vectorstores import Chroma

ids = [str(i) for i in range(0, len(chunks))]

vectordb = Chroma.from_documents(chunks, watsonx_embedding, ids=ids)


In [7]:
vectordb._collection.count()

215

In [8]:
for i in range(3):
    print(vectordb._collection.get(ids=str(i)))

{'ids': ['0'], 'embeddings': None, 'documents': ['1.\tCode of Conduct'], 'metadatas': [{'source': 'companypolicies.txt'}]}
{'ids': ['1'], 'embeddings': None, 'documents': ['Our Code of Conduct outlines the fundamental principles and ethical standards that guide every'], 'metadatas': [{'source': 'companypolicies.txt'}]}
{'ids': ['2'], 'embeddings': None, 'documents': ['that guide every member of our organization. We are committed to maintaining a workplace that is'], 'metadatas': [{'source': 'companypolicies.txt'}]}


Similarity Search

In [9]:
#Similarity Search
query = "Email policy"
docs = vectordb.similarity_search(query)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='to this policy. Non-compliance may lead to appropriate disciplinary action, which could include'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='This policy serves as a framework for handling discipline and termination. The organization'),
 Document(metadata={'source': 'companypolicies.txt'}, page_content='Policy Purpose: The Smoking Policy has been established to provide clear guidance and expectations')]

In [10]:
vectordb.similarity_search(query, k = 1)

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.')]

FAISS db example

In [11]:
#FAISS db
from langchain_community.vectorstores import FAISS
faissdb = FAISS.from_documents(chunks, watsonx_embedding, ids=ids)

for i in range(3):
    print(faissdb.docstore.search(str(i)))

page_content='1.	Code of Conduct' metadata={'source': 'companypolicies.txt'}
page_content='Our Code of Conduct outlines the fundamental principles and ethical standards that guide every' metadata={'source': 'companypolicies.txt'}
page_content='that guide every member of our organization. We are committed to maintaining a workplace that is' metadata={'source': 'companypolicies.txt'}


Similarity search

In [13]:
#Similarity search
query = "Email policy"
docs = faissdb.similarity_search(query, k=1)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='internet and email usage, including those related to copyright and data protection.')]

Managing vector store : Adding, updating, and deleting entries

In [14]:
#Managing vector store: Adding, updating, and deleting entries
#Add
text = "Instructlab is the best open source tool for fine-tuning a LLM." #Content
from langchain_core.documents import Document

#Creating one chunck document
new_chunk =  Document(
    page_content=text,
    metadata={
        "source": "ibm.com",
        "page": 1
    }
)

#Adding new chunck to array
new_chunks = [new_chunk]

print(vectordb._collection.get(ids=['215']))


{'ids': [], 'embeddings': None, 'documents': [], 'metadatas': []}


In [15]:
vectordb.add_documents(
    new_chunks,
    ids=["215"]
)

['215']

In [16]:
vectordb._collection.count()

216

In [17]:
print(vectordb._collection.get(ids=['215']))

{'ids': ['215'], 'embeddings': None, 'documents': ['Instructlab is the best open source tool for fine-tuning a LLM.'], 'metadatas': [{'source': 'ibm.com', 'page': 1}]}


In [18]:
#Update
#Create new chunk
update_chunk =  Document(
    page_content="Instructlab is a perfect open source tool for fine-tuning a LLM.",
    metadata={
        "source": "ibm.com",
        "page": 1
    }
)
#add at existing location
vectordb.update_document(
    '215',
    update_chunk,
)

print(vectordb._collection.get(ids=['215']))

{'ids': ['215'], 'embeddings': None, 'documents': ['Instructlab is a perfect open source tool for fine-tuning a LLM.'], 'metadatas': [{'source': 'ibm.com', 'page': 1}]}


In [19]:
#Delete
vectordb._collection.delete(ids=['215'])

In [20]:
print(vectordb._collection.get(ids=['215']))

{'ids': [], 'embeddings': None, 'documents': [], 'metadatas': []}


Another similarity search example

In [22]:
#Example similarity search
query = "Smoking policy"
docs = vectordb.similarity_search(query, k=1)
docs

[Document(metadata={'source': 'companypolicies.txt'}, page_content='Smoking Restrictions: Smoking inside company buildings, offices, meeting rooms, and other enclosed')]